In [13]:
# 初回だけ実行
%pip install pandas
%pip install tabulate
# %pip install rp2040-pio-emulator@git+https://github.com/NathanY3G/rp2040-pio-emulator
# %pip install adafruit-circuitpython-pioasm

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.2/11.5 MB 36.5 MB/s eta 0:00:01
   ---------- ----------------------------- 3.0/11.5 MB 38.7 MB/s eta 0:00:01
   ----------------- ---------------------- 4.9/11.5 MB 44.6 MB/s eta 0:00:01
   ----------------------- ---------------- 6.8/11.5 MB 48.1 MB/s eta 0:00:01
   ----------------------------- ---------- 8.5/11.5 MB 49.1 MB/s eta 0:00:01
   ----------------------------------- ---- 10.1/11.5 MB 45.9 MB/s eta 0:00:01
   ---------------------------------------  11.5/11.5 MB 50.4 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 46.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   -----

In [39]:
from typing import List
import pandas as pd
from collections import deque
import itertools
import adafruit_pioasm
import pioemu

# NAND IC Interconnect
# | 31  | 30  | 29  | 28  | 27  | 26  | 25  | 24  | 23  | 22  | 21  | 20  | 19  | 18  | 17  | 16  | 15  | 14  | 13  | 12  | 11  | 10  | 9    | 8    | 7   | 6   | 5   | 4   | 3   | 2   | 1   | 0   |
# | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---- | ---- | --- | --- | --- | --- | --- | --- | --- | --- |
# | --  | --  | --  | --  | --  | --  | led | --  | --  | --  | --  | --  | --  | --  | --  | --  | rbb | reb | web | wpb | ale | cle | ceb1 | ceb0 | io7 | io6 | io5 | io4 | io3 | io2 | io1 | io0 |


# PIO Assembler
program = """
.program pio-nandio-readid
    pull block 
    out pindirs, 32 
loop:
    jmp x-- loop
    wait 1 gpio 0
"""
opcodes = adafruit_pioasm.assemble(program)

# TX FIFO
tx_fifo_entries: List[int] = [
    # pindirs
    0xFFFFFFFF,
]

# emulation
test_cycles = 100
generator = pioemu.emulate(
    opcodes=opcodes,
    stop_when=lambda _, state: state.clock > test_cycles,
    initial_state=pioemu.State(
        clock=0,
        program_counter=0,
        pin_directions=0,
        pin_values=0,
        transmit_fifo=deque(tx_fifo_entries),
        receive_fifo=deque([]),
    ),
)

# run
run_states = [
    after.__dict__ for before, after in itertools.islice(generator, test_cycles)
]
run_states_df = pd.DataFrame.from_records(run_states)
rx_fifo = run_states_df[-1:]["receive_fifo"].values

# 後処理


def to_hex_u32(x: int) -> str:
    """32bit符号なし整数を16進数文字列に変換する"""
    return f"0x{(x& 0xffffffff):08X}"


insert_idx = 0


def get_insert_idx() -> int:
    """DataFrameに新しい列を挿入するためのインデックスを返す"""
    global insert_idx
    dst = insert_idx # increment前を返す
    insert_idx += 1
    return dst

run_states_df.insert(
    get_insert_idx(),
    "cyc",
    run_states_df["clock"],
)
run_states_df.insert(
    get_insert_idx(),
    "pc",
    run_states_df["program_counter"],
)
run_states_df.insert(
    get_insert_idx(),
    "inst",
    run_states_df["pc"].map(lambda pc: to_hex_u32(opcodes[pc])),
)
run_states_df.insert(get_insert_idx(), "x", run_states_df["x_register"].map(to_hex_u32))
run_states_df.insert(get_insert_idx(), "y", run_states_df["y_register"].map(to_hex_u32))
run_states_df.insert(
    get_insert_idx(), "pindirs", run_states_df["pin_directions"].map(to_hex_u32)
)
run_states_df.insert(get_insert_idx(), "io", run_states_df["pin_values"] & 0x000000FF)
run_states_df.insert(get_insert_idx(), "io_hex", run_states_df["io"].map(to_hex_u32))
run_states_df.insert(get_insert_idx(), "ceb0", run_states_df["pin_values"] & 0x00000100)
run_states_df.insert(get_insert_idx(), "ceb1", run_states_df["pin_values"] & 0x00000200)
run_states_df.insert(get_insert_idx(), "cle", run_states_df["pin_values"] & 0x00000400)
run_states_df.insert(get_insert_idx(), "ale", run_states_df["pin_values"] & 0x00000800)
run_states_df.insert(get_insert_idx(), "wpb", run_states_df["pin_values"] & 0x00001000)
run_states_df.insert(get_insert_idx(), "web", run_states_df["pin_values"] & 0x00002000)
run_states_df.insert(get_insert_idx(), "reb", run_states_df["pin_values"] & 0x00004000)
run_states_df.insert(get_insert_idx(), "rbb", run_states_df["pin_values"] & 0x00008000)
run_states_df.insert(get_insert_idx(), "led", run_states_df["pin_values"] & 0x01000000)

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

print(f"rx_fifo: {rx_fifo}")
run_states_df


rx_fifo: [deque([])]


,cyc,pc,inst,x,y,pindirs,io,io_hex,ceb0,ceb1,cle,ale,wpb,web,reb,rbb,led,clock,program_counter,pin_directions,pin_values,transmit_fifo,receive_fifo,input_shift_register,output_shift_register,x_register,y_register
0,1,1,0x00006080,0x00000000,0x00000000,0x00000000,0,0x00000000,0,0,0,0,0,0,0,0,0,1,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=4294967295, counter=0)",0,0
1,2,2,0x00000042,0x00000000,0x00000000,0xFFFFFFFF,0,0x00000000,0,0,0,0,0,0,0,0,0,2,2,4294967295,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",0,0
2,3,3,0x00002080,0xFFFFFFFF,0x00000000,0xFFFFFFFF,0,0x00000000,0,0,0,0,0,0,0,0,0,3,3,4294967295,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",-1,0
3,4,3,0x00002080,0xFFFFFFFF,0x00000000,0xFFFFFFFF,0,0x00000000,0,0,0,0,0,0,0,0,0,4,3,4294967295,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",-1,0
4,5,3,0x00002080,0xFFFFFFFF,0x00000000,0xFFFFFFFF,0,0x00000000,0,0,0,0,0,0,0,0,0,5,3,4294967295,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",-1,0
5,6,3,0x00002080,0xFFFFFFFF,0x00000000,0xFFFFFFFF,0,0x00000000,0,0,0,0,0,0,0,0,0,6,3,4294967295,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",-1,0
6,7,3,0x00002080,0xFFFFFFFF,0x00000000,0xFFFFFFFF,0,0x00000000,0,0,0,0,0,0,0,0,0,7,3,4294967295,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",-1,0
7,8,3,0x00002080,0xFFFFFFFF,0x00000000,0xFFFFFFFF,0,0x00000000,0,0,0,0,0,0,0,0,0,8,3,4294967295,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",-1,0
8,9,3,0x00002080,0xFFFFFFFF,0x00000000,0xFFFFFFFF,0,0x00000000,0,0,0,0,0,0,0,0,0,9,3,4294967295,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",-1,0
9,10,3,0x00002080,0xFFFFFFFF,0x00000000,0xFFFFFFFF,0,0x00000000,0,0,0,0,0,0,0,0,0,10,3,4294967295,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",-1,0
